In [1]:
import pandas as pd
import numpy as np

## Importar datos

In [2]:
def indexar_df_por_fecha_diaria(ruta_datos):
    # dataframe
    df = pd.read_csv(ruta_datos)

    # poner la fecha en formato datetime
    df['Date'] = pd.to_datetime(df['Date'])
    # establecer como índice del dataframe
    df_indexado = df.set_index('Date')
    # frecuencia diaria
    df_indexado = df_indexado.asfreq('D')

    return df_indexado

In [3]:
# DATOS PRODUCCIÓN HIDRÁULICA EMBALSES, 
df_PHE = pd.read_csv('data_PHE_1980_2024.csv', index_col=0, parse_dates=True)
                                    # tomamos la primer columna del csv como índice  en formato fecha
# DATOS PRODUCCIÓN HIDRÁULICA RÍOS
df_PHR = pd.read_csv('data_PHR_1980_2024.csv', index_col=0, parse_dates=True)


# Verificar si los índices (fechas) son iguales entre ambos DataFrames
fechas_coinciden = df_PHE.index.equals(df_PHR.index)
if fechas_coinciden:
    print("Las fechas coinciden perfectamente entre los dos DataFrames.")
else:
    print("Las fechas no coinciden entre los dos DataFrames.")

Las fechas coinciden perfectamente entre los dos DataFrames.


## Matriz cobertura

In [8]:
matriz_cobertura = pd.read_csv('../MATRIZ_COBERTURA_PAISES.csv', index_col=0)
matriz_cobertura['Total'] = matriz_cobertura.sum(axis=1)
matriz_interes = matriz_cobertura[(matriz_cobertura['Total']==4) | (matriz_cobertura['Total']==5)]
matriz_interes

,DE,PHE,PHR,PET,PEM,Total
Paises,,,,,,
AT,1,1,1,1,0,4
CH,1,1,1,1,0,4
DE,1,1,1,1,0,4
ES,1,1,1,1,0,4
FI,1,0,1,1,1,4
FR,1,1,1,1,1,5
IT,1,1,1,1,1,5
NO,1,1,1,1,1,5
PT,1,1,1,1,0,4


## Producción hidráulica países de interés

In [9]:
lista_PAISES_INTERES = matriz_interes.index.to_list()
print(f'Países interés = {lista_PAISES_INTERES}')

Países interés = ['AT', 'CH', 'DE', 'ES', 'FI', 'FR', 'IT', 'NO', 'PT', 'RO', 'SE', 'SK']


In [20]:
dict_df_prod_hidraulica = {
    'PHE': df_PHE,
    'PHR': df_PHR,
}

# Asegúrate de que todas las series de datos tengan la misma fecha como índice
# Vamos a crear un DataFrame vacío para las fechas
fechas = df_PHE.index  # Usamos el índice de uno de los DataFrames que ya tiene fechas


In [ ]:
df_prod_hidr_diaria = pd.DataFrame(index=fechas)
dict_paises_coberturas = {}

# Iteramos por los países de interés para crear una serie temporal de producción total
for pais in lista_PAISES_INTERES:

    # dataframe de tipos de energía producidas por cada país
    df_pais = pd.DataFrame(index=fechas)
    
    # para cada cobertura, obtenemos la producción del país para ese día
    for cob, df_cob in dict_df_prod_hidraulica.items():
        if pais in df_cob.columns:  # si el país está en los datos
            df_pais[cob] = df_cob[pais]  # guardamos todas las producciones
        
        elif (pais not in df_cob.columns) and (pais in ['AT','CH','SK']): # países sin costa de interés AT, CH, SK
            # lo dejamos en blanco
            df_pais[cob] = np.nan
        
        else:
            raise ValueError(f'El país {pais} no está en la bbdd {cob}.')

    # suma diaria de produccion
    df_pais['Prod_hidr_diaria'] = df_pais.sum(axis=1)
     
    # añadimos los totales al dataframe de produccion total por países
    df_prod_hidr_diaria[pais] = df_pais['Prod_hidr_diaria']

    # guardamos el dataframe en el diccionario
    dict_paises_coberturas[pais] = df_pais


In [22]:
dict_paises_coberturas['AT'].tail()

,PHE,PHR,Prod_hidr_diaria
Date,,,
2025-02-24,7333.36,39405.36,46738.72
2025-02-25,7452.46,44596.26,52048.72
2025-02-26,7345.51,48103.23,55448.74
2025-02-27,7361.28,47558.24,54919.52
2025-02-28,7337.01,49796.12,57133.13


In [23]:
df_prod_hidr_diaria.tail()

,AT,CH,DE,ES,FI,FR,IT,NO,PT,RO,SE,SK
Date,,,,,,,,,,,,
2025-02-24,46738.72,14247.02,34310.28,96224.72,35751.23,172825.04,82861.87,366209.36,36717.20,31089.23,194219.85,14037.47
2025-02-25,52048.72,13464.07,34517.66,96955.45,35339.95,179020.59,82275.08,392735.75,37908.73,31195.81,194209.67,14167.60
2025-02-26,55448.74,16751.24,34349.44,94943.12,35653.25,185439.32,86329.60,392733.83,36500.42,31163.23,198838.40,14847.47
2025-02-27,54919.52,17619.43,34237.79,94277.65,36716.72,190770.72,83111.47,401839.48,37646.42,33007.17,200144.28,15890.60
2025-02-28,57133.13,18660.22,34408.11,94094.63,37152.60,191046.30,81670.37,395099.02,39964.70,33342.53,204220.67,16140.91


In [25]:
# Crear un DataFrame vacío para la producción anual
df_prod_hidr_anual = pd.DataFrame()

# Agrupar por año y sumar la producción para cada país
df_prod_hidr_anual = df_prod_hidr_diaria.resample('Y').sum()

df_prod_hidr_anual.to_csv('calc_dato_Produccion_Hidraulica_Anual_MWh_paises_interes.csv')